# Scrape "Big 5" European league data from FBref.com

Source Cited - most of this specific FBref scraping file was taken from this GitHub repository:

https://github.com/parth1902/Scrape-FBref-data/blob/master/Scrape_FBref.ipynb

In turn, that file gives credit for much of its work originally coming from another GitHub repository:

https://github.com/chmartin/FBref_EPL

In [1]:
import pandas as pd
import numpy as np
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
from bs4 import BeautifulSoup
import lxml
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import requests
import re
import sys, getopt
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import os
import csv
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# import sqlalchemy

## define LIST variables for all COLUMN NAMES within each FBref table

In [2]:
#standard(stats)

stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#goalkeeping(keepers)

keepers = ["player","nationality","position","squad","age","birth_year","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]

keepers3 = ["players_used","games_gk","games_starts_gk","minutes_gk","goals_against_gk","goals_against_per90_gk","shots_on_target_against","saves","save_pct","wins_gk","draws_gk","losses_gk","clean_sheets","clean_sheets_pct","pens_att_gk","pens_allowed","pens_saved","pens_missed_gk"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#advance goalkeeping(keepersadv)

keepersadv = ["player","nationality","position","squad","age","birth_year","minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]

keepersadv2 = ["minutes_90s","goals_against_gk","pens_allowed","free_kick_goals_against_gk","corner_kick_goals_against_gk","own_goals_against_gk","psxg_gk","psnpxg_per_shot_on_target_against","psxg_net_gk","psxg_net_per90_gk","passes_completed_launched_gk","passes_launched_gk","passes_pct_launched_gk","passes_gk","passes_throws_gk","pct_passes_launched_gk","passes_length_avg_gk","goal_kicks","pct_goal_kicks_launched","goal_kick_length_avg","crosses_gk","crosses_stopped_gk","crosses_stopped_pct_gk","def_actions_outside_pen_area_gk","def_actions_outside_pen_area_per90_gk","avg_distance_def_actions_gk"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#shooting(shooting)

shooting = ["player","nationality","position","squad","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#passing(passing)

passing = ["player","nationality","position","squad","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]

passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#passtypes(passing_types)

passing_types = ["player","nationality","position","squad","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]

passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#goal and shot creation(gca)

gca = ["player","nationality","position","squad","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_og_for"]

gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_og_for"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#defensive actions(defense)

defense = ["player","nationality","position","squad","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]

defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#possession(possession)

possession = ["player","nationality","position","squad","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]

possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#playingtime(playingtime)

playingtime = ["player","nationality","position","squad","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]

playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

#miscallaneous(misc)

misc = ["player","nationality","position","squad","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

# FUNCTIONS to load FBref tables into dataframes using BeautifulSoup

In [3]:
def get_tables(url):
    
    res = requests.get(url)
    
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    
    all_tables = soup.findAll("tbody")
    
    team_table = all_tables[0]
    
    player_table = all_tables[1]
    
    return player_table, team_table

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def get_frame(features, player_table):
    
    pre_df_player = dict()
    
    features_wanted_player = features
    
    rows_player = player_table.find_all('tr')
    
    for row in rows_player:
        
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
            
                cell = row.find("td",{"data-stat": f})
                
                if cell is None:
                    text = "NONE"
                
                else:
                    a = cell.text.strip().encode()
                
                    text=a.decode("utf-8")
                
                if(text == ''):
                    text = 'blank'  # '0'
                    
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                    
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                    
                else:
                    pre_df_player[f] = [text]
                    
    df_player = pd.DataFrame.from_dict(pre_df_player)
    
    return df_player

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def get_frame_team(features, team_table):
    
    pre_df_squad = dict()
    
    #Note: features does not contain squad name, it requires special treatment
    
    features_wanted_squad = features
    
    rows_squad = team_table.find_all('tr')
    
    for row in rows_squad:
        
        if(row.find('th',{"scope":"row"}) != None):
            
            name = row.find('th',{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
                
            else:
                pre_df_squad['squad'] = [name]
                
            for f in features_wanted_squad:
                
                cell = row.find("td",{"data-stat": f})
                
                
                
                a = cell.text.strip().encode()
                
                text=a.decode("utf-8")
                
                
                
                if(text == ''):
                    text = '0'
                    
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                    
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                    
                else:
                    pre_df_squad[f] = [text]
                    
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    
    return df_squad

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def frame_for_category(category,top,end,features):
    
    url = (top + category + end)
    
    player_table, team_table = get_tables(url)
    
    df_player = get_frame(features, player_table)
    
    return df_player

#- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def frame_for_category_team(category,top,end,features):
    
    url = (top + category + end)
    
    player_table, team_table = get_tables(url)
    
    df_team = get_frame_team(features, team_table)
    
    return df_team

# FUNCTION to get the data for outfield PLAYERS

NOTE: "outfield" means "non-goalkeeper"

## includes all categories:

- standard stats

- shooting

- passing

- passing types

- goal and shot creation

- defensive actions

- possession

- miscallaneous

In [4]:
def get_outfield_data(top, end):
    
    df1 = frame_for_category('stats',top,end,stats)
    
    df2 = frame_for_category('shooting',top,end,shooting2)
    
    df3 = frame_for_category('passing',top,end,passing2)
    
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    
    df5 = frame_for_category('gca',top,end,gca2)
    
    df6 = frame_for_category('defense',top,end,defense2)
    
    df7 = frame_for_category('possession',top,end,possession2)
    
    df8 = frame_for_category('misc',top,end,misc2)
    
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    
    df = df.loc[:,~df.columns.duplicated()]
    
    return df

# FUNCTION to get GOALKEEPING and "Advanced GK" data

In [5]:
def get_keeper_data(top,end):
    
    df1 = frame_for_category('keepers',top,end,keepers)
    
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    
    df = pd.concat([df1, df2], axis=1)
    
    df = df.loc[:,~df.columns.duplicated()]
    
    return df

# FUNCTION to get data for entire TEAMS

### Covers all of the same categories as what FBref provides for individual players

In [6]:
def get_team_data(top,end):
    
    df1 = frame_for_category_team('stats',top,end,stats3)
    
    df2 = frame_for_category_team('keepers',top,end,keepers3)
    
    df3 = frame_for_category_team('keepersadv',top,end,keepersadv2)
    
    df4 = frame_for_category_team('shooting',top,end,shooting3)
    
    df5 = frame_for_category_team('passing',top,end,passing2)
    
    df6 = frame_for_category_team('passing_types',top,end,passing_types2)
    
    df7 = frame_for_category_team('gca',top,end,gca2)
    
    df8 = frame_for_category_team('defense',top,end,defense2)
    
    df9 = frame_for_category_team('possession',top,end,possession2)
    
    df10 = frame_for_category_team('misc',top,end,misc2)
    
    #df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=1)
    
    #df = df.loc[:,~df.columns.duplicated()]
    
    #return df
    
    return df1, df2, df3, df4, df5, df6, df7, df8, df9, df10

# CODE that actually uses these functions...

# TO LOAD the up-to-date FBref DATA:

## CODE to actually LOAD data for outfield PLAYERS:

### (1) go to the 'Standard stats' page of the league

- for the English Premier League, the link is...

https://fbref.com/en/comps/9/stats/Premier-League-Stats


- for the Italian league Serie A, the link is...

https://fbref.com/en/comps/11/stats/Serie-A-Stats


### (2) remove the 'stats' part of that URL

### (3) then pass the first and third part of the URL as parameters like below

In [7]:
#EPL_df_PLAYERS = get_outfield_data('https://fbref.com/en/comps/9/','/Premier-League-Stats')
#EPL_df_PLAYERS

In [8]:
# EPL_df_PLAYERS_transposed = EPL_df_PLAYERS.transpose()

# EPL_df_PLAYERS_transposed

In [9]:
# save that data locally in a CSV file:

# EPL_df_PLAYERS.to_csv('Premier_League_20_21_outfield_PLAYERS.csv',index=False)

In [10]:
#SERIE_A_df_PLAYERS = get_outfield_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')
#SERIE_A_df_PLAYERS

## CODE to actually LOAD data for entire TEAMS:

In [11]:
#Serie_A_df_TEAMS = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

#Serie_A_df_TEAMS

In [12]:
#columns_list = list(Serie_A_df_TEAMS.columns)

#columns_list

In [13]:
df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

In [14]:
main_stats_df = df1

main_stats_df

,squad,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,goals_pens_per90,goals_assists_pens_per90,xg,npxg,xa,xg_per90,xa_per90,xg_xa_per90,npxg_per90,npxg_xa_per90
0,Atalanta,30.0,54.2,33.0,363.0,2970.0,77.0,55.0,6.0,6.0,...,2.15,3.82,66.8,62.4,48.2,2.02,1.46,3.49,1.89,3.35
1,Benevento,26.0,41.8,33.0,363.0,2970.0,37.0,23.0,5.0,7.0,...,0.97,1.67,36.7,31.2,22.9,1.11,0.69,1.81,0.95,1.64
2,Bologna,33.0,50.0,33.0,363.0,2970.0,42.0,33.0,2.0,3.0,...,1.21,2.21,44.0,41.9,31.2,1.33,0.95,2.28,1.27,2.21
3,Cagliari,32.0,44.3,33.0,363.0,2970.0,39.0,26.0,4.0,5.0,...,1.06,1.85,38.6,34.9,25.3,1.21,0.79,2.00,1.09,1.88
4,Crotone,29.0,47.0,33.0,363.0,2970.0,41.0,24.0,9.0,9.0,...,0.97,1.70,34.4,27.7,21.7,1.04,0.66,1.70,0.84,1.50
5,Fiorentina,30.0,47.2,33.0,363.0,2970.0,39.0,27.0,5.0,5.0,...,1.03,1.85,40.2,36.4,26.7,1.22,0.81,2.03,1.10,1.91
6,Genoa,33.0,45.1,33.0,363.0,2970.0,37.0,27.0,1.0,1.0,...,1.09,1.91,30.3,29.5,21.0,0.92,0.64,1.56,0.89,1.53
7,Hellas Verona,30.0,49.4,33.0,363.0,2970.0,33.0,25.0,3.0,3.0,...,0.91,1.67,37.5,35.2,26.7,1.14,0.81,1.94,1.07,1.87
8,Inter,23.0,52.1,33.0,363.0,2970.0,68.0,51.0,5.0,6.0,...,1.91,3.45,64.0,59.4,46.8,1.94,1.42,3.36,1.80,3.22
9,Juventus,28.0,58.5,33.0,363.0,2970.0,64.0,52.0,6.0,7.0,...,1.76,3.33,63.6,58.5,45.8,1.93,1.39,3.32,1.77,3.16


In [15]:
match_week = str(int(main_stats_df.games.max()))

match_week

'33'

In [16]:
# new_folder_path = ('~/SBNation_Articles_Soccer_Data/' +
#                    'Serie_A_full_league_2020_21/' +
#                    'TEAM_stats_after_each_Match_Week/'
#                   )

# new_folder_path

In [17]:
# new_folder_name = ('after_MW_' + match_week + '/')

# new_folder_name

In [18]:
# path = os.path.join(new_folder_path, new_folder_name)

In [19]:
# print(path)

In [20]:
# os.makedirs(path)

In [21]:
# os.path.exists(path)

In [22]:
# os.path.isdir(path)

In [23]:
# os.listdir(new_folder_path)

In [24]:
# main_stats_df.to_csv((path + "main_stats.csv"),
#                      index=False)

In [25]:
# main_stats_df.to_csv("main_stats.csv",
#                      index=False)

In [26]:
keepers_df = df2

keepers_df

,squad,players_used,games_gk,games_starts_gk,minutes_gk,goals_against_gk,goals_against_per90_gk,shots_on_target_against,saves,save_pct,wins_gk,draws_gk,losses_gk,clean_sheets,clean_sheets_pct,pens_att_gk,pens_allowed,pens_saved,pens_missed_gk
0,Atalanta,2.0,33.0,33.0,2970.0,39.0,1.18,98.0,59.0,61.2,20.0,8.0,5.0,10.0,30.3,4.0,1.0,3.0,0.0
1,Benevento,1.0,33.0,33.0,2970.0,66.0,2.00,150.0,92.0,58.7,7.0,10.0,16.0,7.0,21.2,5.0,4.0,1.0,0.0
2,Bologna,3.0,33.0,33.0,2970.0,53.0,1.61,165.0,117.0,70.9,10.0,8.0,15.0,5.0,15.2,7.0,5.0,2.0,0.0
3,Cagliari,2.0,33.0,33.0,2970.0,56.0,1.70,178.0,123.0,71.3,8.0,7.0,18.0,5.0,15.2,5.0,5.0,0.0,0.0
4,Crotone,1.0,33.0,33.0,2970.0,83.0,2.52,186.0,105.0,58.1,5.0,3.0,25.0,2.0,6.1,5.0,5.0,0.0,0.0
5,Fiorentina,2.0,33.0,33.0,2970.0,54.0,1.64,121.0,76.0,62.0,8.0,10.0,15.0,6.0,18.2,10.0,8.0,2.0,0.0
6,Genoa,3.0,33.0,33.0,2969.0,48.0,1.46,150.0,105.0,72.0,8.0,12.0,13.0,7.0,21.2,6.0,6.0,0.0,0.0
7,Hellas Verona,1.0,33.0,33.0,2970.0,41.0,1.24,123.0,88.0,71.5,10.0,8.0,14.0,6.0,18.2,7.0,6.0,0.0,1.0
8,Inter,1.0,33.0,33.0,2970.0,29.0,0.88,98.0,72.0,73.5,24.0,7.0,2.0,13.0,39.4,4.0,3.0,1.0,0.0
9,Juventus,2.0,33.0,33.0,2970.0,30.0,0.91,99.0,73.0,74.7,19.0,9.0,5.0,8.0,24.2,6.0,5.0,1.0,0.0


In [27]:
adv_keepers_df = df3

adv_keepers_df

,squad,minutes_90s,goals_against_gk,pens_allowed,free_kick_goals_against_gk,corner_kick_goals_against_gk,own_goals_against_gk,psxg_gk,psnpxg_per_shot_on_target_against,psxg_net_gk,...,passes_length_avg_gk,goal_kicks,pct_goal_kicks_launched,goal_kick_length_avg,crosses_gk,crosses_stopped_gk,crosses_stopped_pct_gk,def_actions_outside_pen_area_gk,def_actions_outside_pen_area_per90_gk,avg_distance_def_actions_gk
0,Atalanta,33.0,39.0,1.0,0.0,5.0,1.0,34.3,0.31,-3.7,...,33.7,156.0,67.3,51.7,198.0,11.0,5.6,33.0,1.00,16.4
1,Benevento,33.0,66.0,4.0,0.0,10.0,6.0,56.7,0.35,-3.3,...,41.0,250.0,60.8,45.2,350.0,30.0,8.6,16.0,0.48,14.2
2,Bologna,33.0,53.0,5.0,2.0,7.0,2.0,52.3,0.30,1.3,...,32.1,262.0,29.0,25.7,277.0,23.0,8.3,22.0,0.67,13.7
3,Cagliari,33.0,56.0,5.0,1.0,8.0,0.0,54.1,0.29,0.1,...,37.0,254.0,58.7,42.6,308.0,30.0,9.7,15.0,0.47,12.9
4,Crotone,33.0,83.0,5.0,2.0,10.0,1.0,72.2,0.37,-9.8,...,34.9,282.0,37.2,32.1,284.0,17.0,6.0,29.0,0.88,15.3
5,Fiorentina,33.0,54.0,8.0,1.0,11.0,1.0,50.9,0.35,-2.1,...,41.3,249.0,55.4,44.2,308.0,21.0,6.8,16.0,0.48,14.4
6,Genoa,33.0,48.0,6.0,0.0,7.0,2.0,45.5,0.28,-0.5,...,40.4,287.0,47.7,38.5,337.0,16.0,4.7,10.0,0.30,12.6
7,Hellas Verona,33.0,41.0,6.0,1.0,7.0,2.0,37.2,0.27,-1.8,...,47.0,267.0,90.3,59.0,280.0,9.0,3.2,13.0,0.39,13.2
8,Inter,33.0,29.0,3.0,1.0,3.0,1.0,26.8,0.24,-1.2,...,29.2,167.0,29.3,31.4,265.0,15.0,5.7,14.0,0.42,12.3
9,Juventus,33.0,30.0,5.0,1.0,4.0,1.0,29.6,0.25,0.6,...,28.8,215.0,14.9,23.5,284.0,17.0,6.0,26.0,0.79,14.0


In [28]:
shooting_df = df4

shooting_df

,squad,goals,pens_made,pens_att,shots_total,shots_on_target,shots_free_kicks,shots_on_target_pct,shots_total_per90,shots_on_target_per90,goals_per_shot,goals_per_shot_on_target,xg,npxg,npxg_per_shot,xg_net,npxg_net
0,Atalanta,77.0,6.0,6.0,547.0,195.0,23.0,35.6,16.58,5.91,0.13,0.36,66.8,62.4,0.12,10.2,8.6
1,Benevento,37.0,5.0,7.0,358.0,111.0,13.0,31.0,10.85,3.36,0.09,0.29,36.7,31.2,0.09,0.3,0.8
2,Bologna,42.0,2.0,3.0,415.0,151.0,22.0,36.4,12.58,4.58,0.10,0.26,44.0,41.9,0.10,-2.0,-1.9
3,Cagliari,39.0,4.0,5.0,377.0,110.0,10.0,29.2,11.42,3.33,0.09,0.32,38.6,34.9,0.09,-2.6,-2.9
4,Crotone,41.0,9.0,9.0,315.0,99.0,14.0,31.4,9.55,3.00,0.10,0.32,34.4,27.7,0.09,6.6,4.3
5,Fiorentina,39.0,5.0,5.0,332.0,108.0,12.0,32.5,10.06,3.27,0.10,0.31,40.2,36.4,0.11,-1.2,-2.4
6,Genoa,37.0,1.0,1.0,291.0,98.0,6.0,33.7,8.82,2.97,0.12,0.37,30.3,29.5,0.10,6.7,6.5
7,Hellas Verona,33.0,3.0,3.0,334.0,101.0,11.0,30.2,10.12,3.06,0.09,0.30,37.5,35.2,0.11,-4.5,-5.2
8,Inter,68.0,5.0,6.0,475.0,173.0,11.0,36.4,14.39,5.24,0.13,0.36,64.0,59.4,0.13,4.0,3.6
9,Juventus,64.0,6.0,7.0,510.0,180.0,22.0,35.3,15.45,5.45,0.11,0.32,63.6,58.5,0.12,0.4,-0.5


In [29]:
passing_df = df5

passing_df

,squad,passes_completed,passes,passes_pct,passes_total_distance,passes_progressive_distance,passes_completed_short,passes_short,passes_pct_short,passes_completed_medium,...,passes_long,passes_pct_long,assists,xa,xa_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes
0,Atalanta,15464.0,18714.0,82.6,290435.0,95493.0,6562.0,7383.0,88.9,6692.0,...,3035.0,64.7,55.0,48.2,6.8,424.0,1255.0,448.0,78.0,1644.0
1,Benevento,9979.0,13149.0,75.9,199296.0,70043.0,3824.0,4444.0,86.0,4266.0,...,2991.0,54.4,23.0,22.9,0.1,249.0,636.0,244.0,69.0,861.0
2,Bologna,12682.0,15867.0,79.9,245518.0,86463.0,5103.0,5836.0,87.4,5432.0,...,3054.0,61.6,33.0,31.2,1.8,302.0,921.0,238.0,54.0,1121.0
3,Cagliari,11181.0,14287.0,78.3,219665.0,76998.0,4370.0,5060.0,86.4,4917.0,...,2832.0,59.0,26.0,25.3,-2.3,256.0,761.0,258.0,75.0,928.0
4,Crotone,11902.0,15025.0,79.2,236150.0,83374.0,4402.0,5044.0,87.3,5602.0,...,2972.0,57.4,24.0,21.7,2.3,233.0,702.0,210.0,61.0,954.0
5,Fiorentina,12152.0,15248.0,79.7,244123.0,79652.0,4662.0,5330.0,87.5,5312.0,...,3203.0,61.4,27.0,26.7,0.3,233.0,940.0,236.0,95.0,992.0
6,Genoa,12249.0,15585.0,78.6,244074.0,78891.0,4848.0,5507.0,88.0,5216.0,...,3372.0,58.0,27.0,21.0,6.0,215.0,762.0,196.0,57.0,854.0
7,Hellas Verona,11523.0,15385.0,74.9,221993.0,75423.0,4749.0,5597.0,84.8,4918.0,...,3156.0,51.2,25.0,26.7,-1.7,253.0,860.0,277.0,60.0,1108.0
8,Inter,15945.0,18707.0,85.2,307461.0,92035.0,6269.0,6956.0,90.1,7224.0,...,3068.0,70.8,51.0,46.8,4.2,361.0,1163.0,379.0,91.0,1350.0
9,Juventus,17705.0,20422.0,86.7,347574.0,101809.0,6552.0,7263.0,90.2,8415.0,...,3420.0,73.3,52.0,45.8,6.2,396.0,1258.0,337.0,85.0,1366.0


In [30]:
pass_types_df = df6

pass_types_df

,squad,passes,passes_live,passes_dead,passes_free_kicks,through_balls,passes_pressure,passes_switches,crosses,corner_kicks,...,passes_left_foot,passes_right_foot,passes_head,throw_ins,passes_other_body,passes_completed,passes_offsides,passes_oob,passes_intercepted,passes_blocked
0,Atalanta,18714.0,17265.0,1449.0,442.0,33.0,2545.0,455.0,375.0,180.0,...,6643.0,10247.0,615.0,599.0,197.0,15464.0,44.0,240.0,420.0,439.0
1,Benevento,13149.0,11486.0,1663.0,591.0,28.0,1835.0,358.0,342.0,142.0,...,4199.0,7213.0,620.0,565.0,219.0,9979.0,47.0,243.0,269.0,336.0
2,Bologna,15867.0,14266.0,1601.0,464.0,28.0,1856.0,410.0,343.0,169.0,...,3461.0,10486.0,677.0,615.0,242.0,12682.0,58.0,230.0,329.0,414.0
3,Cagliari,14287.0,12689.0,1598.0,489.0,12.0,1756.0,331.0,436.0,142.0,...,3275.0,9269.0,499.0,619.0,238.0,11181.0,43.0,248.0,306.0,353.0
4,Crotone,15025.0,13403.0,1622.0,541.0,10.0,2239.0,322.0,331.0,120.0,...,4200.0,9173.0,545.0,564.0,210.0,11902.0,34.0,258.0,321.0,363.0
5,Fiorentina,15248.0,13571.0,1677.0,536.0,18.0,1806.0,523.0,420.0,149.0,...,3728.0,9740.0,526.0,656.0,239.0,12152.0,51.0,222.0,242.0,410.0
6,Genoa,15585.0,14020.0,1565.0,474.0,19.0,1863.0,449.0,303.0,107.0,...,3785.0,9968.0,655.0,616.0,210.0,12249.0,65.0,230.0,282.0,382.0
7,Hellas Verona,15385.0,13760.0,1625.0,494.0,17.0,2285.0,332.0,405.0,155.0,...,5242.0,8031.0,760.0,636.0,167.0,11523.0,87.0,310.0,352.0,451.0
8,Inter,18707.0,17303.0,1404.0,395.0,23.0,2393.0,487.0,420.0,182.0,...,4627.0,12464.0,517.0,569.0,227.0,15945.0,47.0,183.0,386.0,404.0
9,Juventus,20422.0,18997.0,1425.0,430.0,39.0,2331.0,508.0,458.0,198.0,...,6552.0,12386.0,468.0,502.0,256.0,17705.0,70.0,211.0,388.0,362.0


In [31]:
gca_sca_df = df7

gca_sca_df

,squad,sca,sca_per90,sca_passes_live,sca_passes_dead,sca_dribbles,sca_shots,sca_fouled,gca,gca_per90,gca_passes_live,gca_passes_dead,gca_dribbles,gca_shots,gca_fouled,gca_og_for
0,Atalanta,915.0,27.73,699.0,57.0,57.0,46.0,40.0,131.0,3.97,98.0,7.0,12.0,6.0,5.0,0.0
1,Benevento,531.0,16.09,371.0,56.0,26.0,33.0,31.0,55.0,1.67,34.0,7.0,3.0,4.0,5.0,0.0
2,Bologna,656.0,19.88,472.0,51.0,40.0,38.0,38.0,66.0,2.00,49.0,5.0,1.0,6.0,3.0,0.0
3,Cagliari,552.0,17.25,395.0,54.0,26.0,38.0,27.0,58.0,1.81,38.0,3.0,2.0,9.0,5.0,0.0
4,Crotone,518.0,15.70,365.0,44.0,44.0,19.0,30.0,69.0,2.09,38.0,7.0,9.0,4.0,8.0,0.0
5,Fiorentina,499.0,15.12,364.0,46.0,27.0,21.0,34.0,66.0,2.00,47.0,2.0,3.0,7.0,7.0,0.0
6,Genoa,438.0,13.27,333.0,27.0,24.0,19.0,19.0,49.0,1.48,37.0,3.0,2.0,4.0,1.0,0.0
7,Hellas Verona,526.0,15.94,377.0,57.0,25.0,21.0,38.0,64.0,1.94,46.0,2.0,5.0,5.0,6.0,0.0
8,Inter,761.0,23.06,587.0,61.0,32.0,36.0,31.0,114.0,3.45,90.0,7.0,3.0,7.0,6.0,0.0
9,Juventus,860.0,26.06,624.0,72.0,61.0,47.0,41.0,117.0,3.55,87.0,4.0,7.0,10.0,5.0,0.0


In [32]:
defense_df = df8

defense_df

,squad,tackles,tackles_won,tackles_def_3rd,tackles_mid_3rd,tackles_att_3rd,dribble_tackles,dribbles_vs,dribble_tackles_pct,dribbled_past,...,pressures_def_3rd,pressures_mid_3rd,pressures_att_3rd,blocks,blocked_shots,blocked_shots_saves,blocked_passes,interceptions,clearances,errors
0,Atalanta,597.0,375.0,245.0,262.0,90.0,154.0,482.0,32.0,328.0,...,1289.0,2322.0,1278.0,487.0,83.0,3.0,404.0,444.0,684.0,7.0
1,Benevento,517.0,314.0,297.0,167.0,53.0,189.0,538.0,35.1,349.0,...,1650.0,1807.0,758.0,500.0,122.0,1.0,378.0,348.0,777.0,13.0
2,Bologna,506.0,291.0,261.0,180.0,65.0,156.0,464.0,33.6,308.0,...,1688.0,1891.0,1083.0,593.0,141.0,2.0,452.0,391.0,783.0,14.0
3,Cagliari,522.0,320.0,250.0,206.0,66.0,162.0,468.0,34.6,306.0,...,1656.0,2067.0,951.0,504.0,133.0,1.0,371.0,319.0,751.0,6.0
4,Crotone,531.0,325.0,291.0,172.0,68.0,178.0,471.0,37.8,293.0,...,1665.0,1868.0,898.0,503.0,119.0,1.0,384.0,291.0,732.0,15.0
5,Fiorentina,516.0,324.0,261.0,186.0,69.0,192.0,495.0,38.8,303.0,...,1496.0,1978.0,982.0,445.0,120.0,1.0,325.0,327.0,654.0,6.0
6,Genoa,531.0,327.0,272.0,191.0,68.0,168.0,501.0,33.5,333.0,...,1765.0,2193.0,1022.0,520.0,110.0,0.0,410.0,370.0,867.0,11.0
7,Hellas Verona,472.0,291.0,236.0,188.0,48.0,139.0,390.0,35.6,251.0,...,1372.0,1960.0,985.0,530.0,155.0,1.0,375.0,405.0,895.0,14.0
8,Inter,485.0,294.0,225.0,181.0,79.0,165.0,442.0,37.3,277.0,...,1421.0,1815.0,1174.0,455.0,97.0,1.0,358.0,314.0,546.0,6.0
9,Juventus,553.0,349.0,247.0,225.0,81.0,166.0,472.0,35.2,306.0,...,1395.0,1842.0,1024.0,476.0,113.0,1.0,363.0,296.0,590.0,5.0


In [33]:
possession_df = df9

possession_df

,squad,touches,touches_def_pen_area,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_pen_area,touches_live_ball,dribbles_completed,dribbles,...,carry_distance,carry_progressive_distance,progressive_carries,carries_into_final_third,carries_into_penalty_area,pass_targets,passes_received,passes_received_pct,miscontrols,dispossessed
0,Atalanta,22833.0,1711.0,6038.0,11012.0,7155.0,1135.0,21434.0,377.0,628.0,...,79831.0,41326.0,1911.0,578.0,204.0,18070.0,15464.0,85.6,446.0,461.0
1,Benevento,16804.0,2286.0,6520.0,7569.0,3714.0,652.0,15207.0,205.0,361.0,...,53027.0,28183.0,969.0,308.0,82.0,12369.0,9979.0,80.7,353.0,311.0
2,Bologna,19882.0,2642.0,6934.0,9177.0,5015.0,745.0,18314.0,336.0,577.0,...,66926.0,34710.0,1309.0,435.0,136.0,15098.0,12682.0,84.0,397.0,392.0
3,Cagliari,17926.0,2202.0,6171.0,8421.0,4551.0,672.0,16377.0,301.0,482.0,...,60519.0,33649.0,1218.0,443.0,93.0,13690.0,11181.0,81.7,362.0,365.0
4,Crotone,18730.0,2725.0,7472.0,8593.0,3774.0,567.0,17168.0,315.0,531.0,...,60171.0,31885.0,1084.0,333.0,93.0,14241.0,11902.0,83.6,456.0,395.0
5,Fiorentina,18859.0,2000.0,5692.0,9371.0,4987.0,664.0,17245.0,280.0,473.0,...,64933.0,34559.0,1339.0,417.0,106.0,14453.0,12152.0,84.1,369.0,473.0
6,Genoa,19345.0,2451.0,7030.0,9363.0,3969.0,515.0,17824.0,245.0,459.0,...,55986.0,29587.0,1106.0,338.0,79.0,14815.0,12249.0,82.7,419.0,331.0
7,Hellas Verona,19208.0,2101.0,6025.0,9400.0,4891.0,711.0,17628.0,280.0,463.0,...,59226.0,32132.0,1236.0,422.0,109.0,14361.0,11523.0,80.2,396.0,370.0
8,Inter,22076.0,2264.0,7005.0,10537.0,5971.0,992.0,20712.0,231.0,394.0,...,78190.0,43590.0,1693.0,486.0,171.0,18057.0,15945.0,88.3,399.0,314.0
9,Juventus,23962.0,2449.0,7484.0,12011.0,6043.0,1046.0,22608.0,372.0,565.0,...,86655.0,47340.0,1828.0,538.0,230.0,19860.0,17705.0,89.1,374.0,370.0


In [34]:
misc_stats_df = df10

misc_stats_df

,squad,cards_yellow,cards_red,cards_yellow_red,fouls,fouled,offsides,crosses,interceptions,tackles_won,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Atalanta,58.0,1.0,1.0,514.0,417.0,50.0,375.0,444.0,375.0,4.0,4.0,1.0,3290.0,630.0,495.0,56.0
1,Benevento,84.0,4.0,1.0,445.0,513.0,53.0,342.0,348.0,314.0,7.0,5.0,6.0,2574.0,490.0,587.0,45.5
2,Bologna,74.0,4.0,2.0,474.0,413.0,69.0,343.0,391.0,291.0,2.0,7.0,2.0,2792.0,562.0,552.0,50.4
3,Cagliari,68.0,3.0,2.0,452.0,423.0,52.0,436.0,319.0,320.0,4.0,5.0,0.0,2640.0,570.0,566.0,50.2
4,Crotone,81.0,3.0,2.0,462.0,445.0,45.0,331.0,291.0,325.0,7.0,5.0,1.0,2724.0,438.0,530.0,45.2
5,Fiorentina,71.0,4.0,0.0,469.0,457.0,58.0,420.0,327.0,324.0,4.0,10.0,1.0,2620.0,502.0,528.0,48.7
6,Genoa,74.0,1.0,0.0,455.0,424.0,73.0,303.0,370.0,327.0,1.0,6.0,2.0,2814.0,475.0,625.0,43.2
7,Hellas Verona,74.0,1.0,0.0,526.0,439.0,93.0,405.0,405.0,291.0,3.0,7.0,2.0,3338.0,752.0,769.0,49.4
8,Inter,53.0,1.0,0.0,420.0,372.0,59.0,420.0,314.0,294.0,4.0,4.0,1.0,2531.0,485.0,390.0,55.4
9,Juventus,69.0,5.0,1.0,431.0,417.0,90.0,458.0,296.0,349.0,6.0,6.0,1.0,2583.0,427.0,353.0,54.7
